In [ ]:
!pip install selenium

In [154]:
import time
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs

In [189]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
driver.implicitly_wait(5)
driver.get('https://nid.naver.com/nidlogin.login')
time.sleep(10)

_cookies = driver.get_cookies()
cookie_dict = {}
for ck in _cookies:
    cookie_dict[ck['name']] = ck['value']
#driver.quit()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11880\2446565818.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11880\2446565818.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


In [190]:
cookie_dict

{'SB_MODE': 'plusdeal',
 'NID_JKL': 'N9/IW1yEe3PgG5pxVn2e8XHgpUW8KsvC36Q6Bt+sPWU=',
 'NID_SES': 'AAABks/VhE9/yI+pmxzNEovrbgnKEIoaI+FI5GX3W4z1ppTJqFj/TkK86/UmCp2AyLO2Q+xTOx2dKaTYMtbQZEGsTBzHWG7FfcBhq2wqA3r3XxSnzFn6uOOCGWkcLAOAMbMGp62BrfXX7yi4MyBrDZB62UYAt+xHIiN5yb0PSaNruwI16eHGa202PszJbgEHtzVvtSCM9NE3tU2d1R9gW5DJZK9DqiGn+rEt1txOviBfxb10u30aMGrKg+yUo/h6Om9je28FwyuUl4acxOfDi+zthROlGSk5i/h7tuVxwVWe5qJSWlNh/AFQt8+RgwjozY4vykDNHhcK5T7gKQPbFdmgQhahxwncWmF/w41D0yjcSZEZ2M2m6m79LSC/kdp5NKwxpHIQIiIErx49vKr6hjKxeYgCaCPzIzAFHGZATyBhdTacipbxPVSkFkh3Lx9jsTmoSgsD6pYTM5utUUnU4xEkvioxX02FZuvxj6ogV7nH8E2+Mj+6hUROos9pHwqB0kruJN+NrtNFmmo7sVuGBdJUVmwTm3cHMYxoasxu0Zzxyu5n',
 'NID_AUT': 'kYT3/k5+OGjC7fEkCQT3kEjfoeVO89cllfAJzMpLm3XHjkmfLqZ167s4yzfr6S4w',
 'NNB': 'YYJCUC2KUBTWG',
 'nid_inf': '1447774884',
 'PM_CK_loc': '65c2aa43f5cd4fc6e776fd8e3b0e242c3adc108fb096d391680fa8041fc04016'}

In [191]:
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.88 Safari/537.36 Edg/107.0.1418.35',
}

session.headers.update(headers)
session.cookies.update(cookie_dict)

In [210]:
payload={
    'redirect_uri':'https://passport.livere.com/v1/auth/naver/callback',
    'client_id':'56vMBuwZrZtIb6B4KQ0W',
    'auth_type':'',
    'response_type':'code'
}
res_auth = session.post('https://nid.naver.com/oauth2.0/authorize', data=payload)

res_text = res_auth.text
nonce = res_text[res_text.index('nonce=')+7:res_text.index('>\nlocation.replace')-1]
link = res_text[res_text.index('replace(')+9:res_text.index('");\n')]
auth_uri = link[:link.index('?')]
data_str = link[link.index('?')+1:]
token = {}

for param in data_str.split('&'):
    if '=' in param:
        token[param[:param.index('=')]] = param[param.index('=')+1:]
    else:
        token[param]=''

In [211]:
payload={
    'response_type':'code',
    'state':'',
    'redirect_uri':'https://passport.livere.com/v1/auth/naver/callback',
    'client_id':'56vMBuwZrZtIb6B4KQ0W',
    'oauth_os':'',
    'inapp_view':'',
    'locale':'ko_KR',
    'oauth_token':token['oauth_token']
    
}
res_redirect = session.post('https://nid.naver.com/oauth2.0/authorize', data=payload)

In [212]:
passport_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.0.0',
    'Upgrade-Insecure-Requests': '1',
    'Host': 'passport.livere.com',
    'Connection': 'keep-alive',
    'Referer': 'https://nid.naver.com/'
}
session.headers.update(passport_headers)

In [213]:
payload={
    'code':token['oauth_token'],
    'state':'',
}
res_token = session.get('https://passport.livere.com/v1/auth/naver/callback', data=payload)

In [221]:
session.headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.0.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'Host': 'passport.livere.com', 'Referer': 'https://nid.naver.com/'}

In [219]:
for i in session.cookies:
    print(i)

<Cookie SB_MODE=plusdeal for />
<Cookie NID_JKL=N9/IW1yEe3PgG5pxVn2e8XHgpUW8KsvC36Q6Bt+sPWU= for />
<Cookie NID_SES=AAABks/VhE9/yI+pmxzNEovrbgnKEIoaI+FI5GX3W4z1ppTJqFj/TkK86/UmCp2AyLO2Q+xTOx2dKaTYMtbQZEGsTBzHWG7FfcBhq2wqA3r3XxSnzFn6uOOCGWkcLAOAMbMGp62BrfXX7yi4MyBrDZB62UYAt+xHIiN5yb0PSaNruwI16eHGa202PszJbgEHtzVvtSCM9NE3tU2d1R9gW5DJZK9DqiGn+rEt1txOviBfxb10u30aMGrKg+yUo/h6Om9je28FwyuUl4acxOfDi+zthROlGSk5i/h7tuVxwVWe5qJSWlNh/AFQt8+RgwjozY4vykDNHhcK5T7gKQPbFdmgQhahxwncWmF/w41D0yjcSZEZ2M2m6m79LSC/kdp5NKwxpHIQIiIErx49vKr6hjKxeYgCaCPzIzAFHGZATyBhdTacipbxPVSkFkh3Lx9jsTmoSgsD6pYTM5utUUnU4xEkvioxX02FZuvxj6ogV7nH8E2+Mj+6hUROos9pHwqB0kruJN+NrtNFmmo7sVuGBdJUVmwTm3cHMYxoasxu0Zzxyu5n for />
<Cookie NID_AUT=kYT3/k5+OGjC7fEkCQT3kEjfoeVO89cllfAJzMpLm3XHjkmfLqZ167s4yzfr6S4w for />
<Cookie NNB=YYJCUC2KUBTWG for />
<Cookie nid_inf=1447774884 for />
<Cookie PM_CK_loc=65c2aa43f5cd4fc6e776fd8e3b0e242c3adc108fb096d391680fa8041fc04016 for />
<Cookie connect.sid=s%3Ae7f183a0-f843-46d4-91df-be034c60250d.Hq9aldhC9M

In [151]:
res = session.get('https://www.gangnam.go.kr/loginLivere.do', data={'code':'1659f60d9afd9b19e9168369'})

In [155]:
danji_soup = bs(res.text, "html.parser")